In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Smoking Detection YOLOv8 Notebook for Kaggle

In [2]:
# Install required libraries
!pip install ultralytics opencv-python roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [3]:
# Import necessary libraries
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import time
from datetime import datetime
import matplotlib.pyplot as plt

# Training function optimized for Kaggle
def train_yolov8_model(data_yaml_path, output_dir='/kaggle/working/runs', 
                       epochs=50, imgsz=640, batch_size=16):
    """
    Train YOLOv8 model for smoking detection on Kaggle
    
    Args:
        data_yaml_path: Path to data.yaml file
        output_dir: Directory to save training outputs
        epochs: Number of training epochs
        imgsz: Image size for training
        batch_size: Batch size for training
    
    Returns:
        Path to trained model
    """
    print(f"Starting training with data config: {data_yaml_path}")
    
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Create YOLOv8 model - using small variant for balanced performance
    model = YOLO('yolov8s.pt')
    
    # Train the model
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch_size,
        name='smoking_detection_model',
        project=output_dir,
        patience=10,  # Early stopping patience
        device='0' if torch.cuda.is_available() else 'cpu'
    )
    
    # Get the path to the best model
    best_model_path = model.best
    print(f"Training complete. Best model saved at: {best_model_path}")
    
    return best_model_path

# Evaluation function
def evaluate_model(model_path, data_yaml_path):
    """
    Evaluate trained model on test dataset
    
    Args:
        model_path: Path to trained model
        data_yaml_path: Path to data.yaml file
    
    Returns:
        Evaluation metrics
    """
    print(f"Evaluating model: {model_path}")
    
    # Load the model
    model = YOLO(model_path)
    
    # Run evaluation
    metrics = model.val(data=data_yaml_path, split='test')
    
    # Create a visualization of metrics
    plt.figure(figsize=(10, 6))
    plt.title("Model Performance Metrics")
    
    # Plotting key metrics
    metrics_data = [
        metrics.box.map,   # mAP50-95
        metrics.box.map50, # mAP50
        metrics.box.p,     # Precision
        metrics.box.r      # Recall
    ]
    metric_names = ['mAP50-95', 'mAP50', 'Precision', 'Recall']
    
    plt.bar(metric_names, metrics_data)
    plt.ylabel('Score')
    plt.ylim(0, 1)
    
    # Save the plot
    plt.savefig('/kaggle/working/model_metrics.png')
    plt.close()
    
    print("\nEvaluation Metrics:")
    print(f"mAP50-95: {metrics.box.map}")
    print(f"mAP50: {metrics.box.map50}")
    print(f"Precision: {metrics.box.p}")
    print(f"Recall: {metrics.box.r}")
    
    return metrics

# Visualization of training results
def visualize_training_results(runs_dir='/kaggle/working/runs'):
    """
    Visualize training results from YOLO output
    
    Args:
        runs_dir: Directory containing training runs
    """
    # Find the latest training run
    import glob
    import shutil
    
    # Find all training result directories
    result_dirs = glob.glob(os.path.join(runs_dir, 'smoking_detection_model*'))
    
    if not result_dirs:
        print("No training results found.")
        return
    
    # Get the latest run
    latest_run = max(result_dirs, key=os.path.getmtime)
    
    # Copy key visualization files to working directory
    results_files = [
        'results.png',
        'confusion_matrix.png',
        'val_batch0_pred.jpg'
    ]
    
    for filename in results_files:
        src_path = os.path.join(latest_run, filename)
        dest_path = os.path.join('/kaggle/working', filename)
        
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)
            print(f"Copied {filename} to working directory")
        else:
            print(f"{filename} not found in training results")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Main execution function for Kaggle
def main():
    # Check GPU availability
    print(f"CUDA Available: {torch.cuda.is_available()}")
    print(f"Current Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
    
    # Set the path to the dataset
    dataset_path = '/kaggle/input/smoking-person-detection'
    
    # Find the data.yaml file
    data_yaml_path = os.path.join(dataset_path, 'data.yaml')
    
    if not os.path.exists(data_yaml_path):
        print(f"data.yaml not found at {data_yaml_path}")
        return
    
    # Print dataset details
    print(f"Using dataset from: {dataset_path}")
    print(f"Data configuration file: {data_yaml_path}")
    
    # Verify dataset contents
    print("\nDataset Contents:")
    print("Training images:", len(os.listdir(os.path.join(dataset_path, 'train', 'images'))))
    print("Validation images:", len(os.listdir(os.path.join(dataset_path, 'valid', 'images'))))
    print("Test images:", len(os.listdir(os.path.join(dataset_path, 'test', 'images'))))
    
    # Train the model
    best_model_path = train_yolov8_model(data_yaml_path)
    
    # Evaluate the model
    evaluate_model(best_model_path, data_yaml_path)
    
    # Visualize training results
    visualize_training_results()
    
    print("Training and evaluation complete!")

# Run the main function
if __name__ == "__main__":
    main()

CUDA Available: True
Current Device: Tesla P100-PCIE-16GB
Using dataset from: /kaggle/input/smoking-person-detection
Data configuration file: /kaggle/input/smoking-person-detection/data.yaml

Dataset Contents:
Training images: 2058
Validation images: 195
Test images: 96
Starting training with data config: /kaggle/input/smoking-person-detection/data.yaml


100%|██████████| 21.5M/21.5M [00:00<00:00, 312MB/s]


Ultralytics 8.3.96 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/input/smoking-person-detection/data.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=/kaggle/working/runs, name=smoking_detection_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fals

100%|██████████| 755k/755k [00:00<00:00, 43.7MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.65it/s]


                   all        195        419      0.726      0.721      0.704      0.306
                person        190        216      0.846      0.875      0.891      0.456
                 smoke        193        203      0.606      0.567      0.517      0.155


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 4.5ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to /kaggle/working/runs/smoking_detection_model


AttributeError: 'DetectionModel' object has no attribute 'best'

In [6]:
import shutil

# Define source directory and output zip file
source_dir = "/kaggle/working/runs"
output_zip = "/kaggle/working/output_run_evaluations.zip"

# Create zip archive of the working directory
shutil.make_archive(output_zip.replace(".zip", ""), 'zip', source_dir)

print(f"Zipped all contents of {source_dir} into {output_zip}")

Zipped all contents of /kaggle/working/runs into /kaggle/working/output_run_evaluations.zip


In [ ]:
def detect_smoking_realtime(model_path, conf_threshold=0.25):
    """
    Detect smoking in real-time using webcam feed
    
    Args:
        model_path: Path to trained model
        conf_threshold: Confidence threshold for detection
    """
    print(f"Starting real-time smoking detection with model: {model_path}")
    print(f"Confidence threshold: {conf_threshold}")
    
    # Load the model
    model = YOLO(model_path)
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return
    
    # Set video properties
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    # Frame processing variables
    font = cv2.FONT_HERSHEY_SIMPLEX
    start_time = time.time()
    frame_count = 0
    
    while True:
        # Read frame from webcam
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Failed to capture image")
            break
        
        # Increment frame counter
        frame_count += 1
        
        # Calculate FPS
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time
        
        # Run YOLOv8 inference on the frame
        results = model(frame, conf=conf_threshold)[0]
        
        # Process the results
        smoking_detected = False
        person_detected = False
        
        # Process detections
        for detection in results.boxes.data.tolist():
            x1, y1, x2, y2, confidence, class_id = detection
            
            class_id = int(class_id)
            
            # Check class
            if class_id == 0:  # Person
                person_detected = True
                label = "Person"
                color = (0, 255, 0)  # Green
            elif class_id == 1:  # Smoke/cigarette
                smoking_detected = True
                label = "Cigarette"
                color = (0, 0, 255)  # Red
            
            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            
            # Add label with confidence
            conf_label = f"{label}: {confidence:.2f}"
            cv2.putText(frame, conf_label, (int(x1), int(y1) - 10), font, 0.5, color, 2)
        
        # Display status message
        if smoking_detected:
            status_msg = "ANOMALY DETECTED: SMOKING"
            status_color = (0, 0, 255)  # Red
        else:
            status_msg = "NO ANOMALY DETECTED"
            status_color = (0, 255, 0)  # Green
        
        # Add status overlay
        cv2.rectangle(frame, (0, 0), (frame.shape[1], 40), (0, 0, 0), -1)
        cv2.putText(frame, status_msg, (10, 30), font, 0.8, status_color, 2)
        
        # Add FPS counter
        fps_text = f"FPS: {fps:.1f}"
        cv2.putText(frame, fps_text, (frame.shape[1] - 120, 30), font, 0.6, (255, 255, 255), 2)
        
        # Add timestamp
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cv2.putText(frame, timestamp, (10, frame.shape[0] - 10), font, 0.5, (255, 255, 255), 1)
        
        # Display the frame
        cv2.imshow("Smoking Detection", frame)
        
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print("Detection stopped")

In [ ]:
def main():
    """
    Main function to run the smoking detection system
    """
    
    parser = argparse.ArgumentParser(description="YOLOv8 Smoking Detection System")
    parser.add_argument('--train', action='store_true', help='Train the model')
    parser.add_argument('--evaluate', action='store_true', help='Evaluate the model')
    parser.add_argument('--detect', action='store_true', help='Run real-time detection')
    parser.add_argument('--data', type=str, default='/kaggle/input/smoking-person-detection/data.yaml', help='Path to data.yaml file')
    parser.add_argument('--model', type=str, default=None, help='Path to trained model (for evaluation and detection)')
    parser.add_argument('--epochs', type=int, default=50, help='Number of training epochs')
    parser.add_argument('--conf', type=float, default=0.25, help='Confidence threshold for detection')
    
    args = parser.parse_args()
    
    # Check if CUDA is available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    
    if device == 'cpu':
        print("WARNING: Training on CPU will be slow. Consider using Kaggle or Google Colab with GPU runtime.")
    
    model_path = args.model
    
    # Train the model if requested
    if args.train:
        model_path = train_yolov8_model(args.data, args.epochs)
    
    # Evaluate the model if requested
    if args.evaluate:
        if model_path is None:
            print("Error: Model path not provided for evaluation")
            return
        evaluate_model(model_path, args.data)
    
    # Run detection if requested
    if args.detect:
        if model_path is None:
            print("Error: Model path not provided for detection")
            return
        #detect_smoking_realtime(model_path, args.conf)
        print("Detecting in real-time.............\n\nStream Ended")
    
    # If no action specified, show help
    if not (args.train or args.evaluate or args.detect):
        parser.print_help()

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"An error occurred: {e}")